# Инструкция по выполнению

<div align="right">
<a href="#placeholder" class="btn btn-default" data-toggle="collapse">Развернуть список</a>
</div>
<div id="placeholder" class="collapse">
<ol>
    <li>Здесь и далее оценка качества будет выполняться с помощью cross_val_score с cv=5 и остальными параметрами по умолчанию. Оцените среднее качество ( .mean() ) и стандартное отклонение ( .std() ) по fold'ам для: а) pipeline из CountVectorizer() и LogisticRegression(), б) pipeline из TfidfVectorizer() и LogisticRegression(). В соответствующем пункте задания выпишите через пробел среднее в п. а, отклонение в п. а, среднее в п.б и отклонение в п. б [ссылка](#1)</li>
    <li>Попробуйте позадавать параметр min_df у CountVectorizer. Оцените качество вашего классикатора с min_df=10 и с min_df=50 [ссылка](#2)</li>
    <li>Поперебирайте классификатор, используемый после CountVectorizer. И vectorizer и классификатор берите с параметрами по умолчанию. Сравните результаты для LogisticRegression, LinearSVC и SGDClassifier. Выпишите в ответе на соответствующий вопрос самое худшее качество из получившихся [ссылка](#3)</li>
    <li>Подготовьте список стоп-слов с помощью nltk.corpus.stopwords.words('english'), посмотрите на его элементы, и передайте его в соответствующий параметр CountVectorizer. В sklearn также предусмотрен свой список английских стоп-слов - для этого нужно задать соответствующий параметр равным строке 'english'. Оцените качество классификатора в одном и другом случае и выпишете сначала качество в первом варианте, затем во втором в соответствующем вопросе [ссылка](#4)</li>
    <li>Попробуйте в CountVectorizer добавить к словам биграммы и измерить качество модели. А затем постройте модель на частотах буквенных n-грамм c n от 3 до 5, указав соответствующее значение параметра ngram_range и параметр analyzer='char_wb'. Полученные два числа запишите через пробел в ответе на соответствующий вопрос [ссылка](#5)</li>
</ol>
</div>

In [34]:
import numpy as np
import scipy as sp

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn import metrics, model_selection
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import movie_reviews

In [5]:
negids = movie_reviews.fileids('neg')
posids = movie_reviews.fileids('pos')

negfeats = [movie_reviews.words(fileids=[f]) for f in negids]
posfeats = [movie_reviews.words(fileids=[f]) for f in posids]

feedbacks = np.append(negfeats, posfeats)
for indx, elm in enumerate(feedbacks):
    feedbacks[indx] = " ".join(elm)

n, p = len(negfeats), len(posfeats)
negative, positive = np.zeros(n), np.ones(p)

classes = np.append(negative, positive)

cv = 5 # from task

### Задание #1 <a id='1'></a>

In [6]:
%%time

vct = CountVectorizer()
clf = LogisticRegression()

pipe = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

score = model_selection.cross_val_score(pipe, feedbacks, classes)

CPU times: user 5.45 s, sys: 52 ms, total: 5.5 s
Wall time: 5.62 s


In [17]:
score.mean(), score.std()

(0.83602165039290777, 0.015309139640513638)

In [11]:
%%time

tct = TfidfVectorizer()

pipe = Pipeline(steps = [
    ('vectorizer', tct),
    ('logistic', clf)
])

score_tf = model_selection.cross_val_score(pipe, feedbacks, classes)

CPU times: user 4.07 s, sys: 32 ms, total: 4.1 s
Wall time: 4.15 s


In [12]:
score_tf.mean(), score_tf.std()

(0.81351111590632552, 0.01035604041249499)

In [20]:
with open('subm/w02s01.txt', 'w+') as f:
    f.write(" ".join([str(score.mean()), str(score.std()), str(score_tf.mean()), str(score_tf.std())]))

### Задание #2 <a id='2'></a>

In [23]:
vct10 = CountVectorizer(min_df=10)
vct50 = CountVectorizer(min_df=50)
clf = LogisticRegression()

pipe10 = Pipeline(steps = [
    ('vectorizer', vct10),
    ('logistic', clf)
])
pipe50 = Pipeline(steps = [
    ('vectorizer', vct50),
    ('logistic', clf)
])

score10 = model_selection.cross_val_score(pipe10, feedbacks, classes)
score50 = model_selection.cross_val_score(pipe50, feedbacks, classes)

In [26]:
with open('subm/w02s02.txt', 'w+') as f:
    f.write(" ".join([str(score10.mean()), str(score50.mean())]))

### Задание #3 <a id='3'></a>

In [29]:
%%time

vct = CountVectorizer()
clf = LogisticRegression()
svc = LinearSVC()
sgd = SGDClassifier()

pipe_clf = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

pipe_svc = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', svc)
])

pipe_sgd = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', sgd)
])

clf_score = model_selection.cross_val_score(pipe_clf, feedbacks, classes)
svc_score = model_selection.cross_val_score(pipe_svc, feedbacks, classes)
sgd_score = model_selection.cross_val_score(pipe_sgd, feedbacks, classes)

CPU times: user 14.9 s, sys: 60 ms, total: 14.9 s
Wall time: 15.1 s


In [41]:
with open('subm/w02s03.txt', 'w+') as f:
    f.write(str(min(clf_score.mean(), svc_score.mean(), sgd_score.mean())))

### Задание #4 <a id='4'></a>

In [36]:
stop = nltk.corpus.stopwords.words('english')

In [38]:
vct = CountVectorizer(stop_words=stop)
clf = LogisticRegression()

pipe = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

score = model_selection.cross_val_score(pipe, feedbacks, classes)
print("{}".format(score.mean()))

0.8270171368973763


In [39]:
vct = CountVectorizer(stop_words='english')
clf = LogisticRegression()

pipe = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

score = model_selection.cross_val_score(pipe, feedbacks, classes)
print("{}".format(score.mean()))

0.8345141548734363


In [42]:
result = [0.8270171368973763, 0.8345141548734363]

with open('subm/w02s04.txt', 'w+') as f:
    f.write(" ".join([str(i) for i in result]))

### Задание #5 <a id='5'></a>

In [44]:
words_bigrams = { x : i for i, x in enumerate(stop) }

In [46]:
vct = CountVectorizer(stop_words=words_bigrams)
clf = LogisticRegression()

pipe = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

score = model_selection.cross_val_score(pipe, feedbacks, classes)
print("{}".format(score.mean()))

0.8270171368973763


In [47]:
vct = CountVectorizer(analyzer='char_wb', ngram_range=(3, 5))
clf = LogisticRegression()

pipe = Pipeline(steps = [
    ('vectorizer', vct),
    ('logistic', clf)
])

score = model_selection.cross_val_score(pipe, feedbacks, classes)
print("{}".format(score.mean()))

0.8205031378684073


In [48]:
result = [0.8270171368973763, 0.8205031378684073]

with open('subm/w02s05.txt', 'w+') as f:
    f.write(" ".join([str(i) for i in result]))